In [1]:
import numpy as np
import pandas as pd
import time
from random import shuffle, random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report, ConfusionMatrixDisplay
from sklearn.metrics import RocCurveDisplay, roc_curve
from sklearn.metrics import PrecisionRecallDisplay, precision_recall_curve
from tqdm import tqdm
import torch
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchmetrics
import tenseal as ts

In [2]:
# Importing, balancing, scaling and splitting the dataset into train and test set
df = pd.read_csv(r"C:\Users\manig\Downloads\Mitacs\top_feature_df_rfr40.csv")
df['Label'] = df['Label'].replace(0, -1)
        
grouped = df.groupby('Label')
df = grouped.apply(lambda x: x.sample(grouped.size().min(), random_state=73).reset_index(drop=True))

x = df.drop(["Label"], axis = 1)
y = df["Label"]
# scaler = preprocessing.MinMaxScaler()
scaler = preprocessing.StandardScaler()
x = scaler.fit_transform(x)

x_train, x_test_20, y_train, y_test_20 = train_test_split(x, y, test_size=0.02, random_state=42)
x_train=torch.from_numpy(x_train).float().unsqueeze(dim=1)
x_test_20=torch.from_numpy(x_test_20).float().unsqueeze(dim=1)
y_train=torch.from_numpy(np.array(y_train)).float().unsqueeze(dim=1)
y_test_20=torch.from_numpy(np.array(y_test_20)).float().unsqueeze(dim=1)
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test_20.shape}")
print(f"y_test shape: {y_test_20.shape}")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
class SVM(nn.Module):
    def __init__(self, n_features):
        super(SVM, self).__init__()
        self.dense_layer_1 = nn.Linear(n_features, 1)

    def forward(self, x):
        output = self.dense_layer_1(x)
        return output 

In [ ]:
def accuracy(X, y, model):
    correct = 0
    for i in range(len(y)):
        y_predicted = model(X[i]).squeeze().detach()
        y_predicted = torch.sign(y_predicted).float()
        if y_predicted == y[i]: 
            correct += 1
    return float(correct)/len(y)

In [ ]:
def train(X, Y, optimizer, epoch, batchsize, step, model):
    loss_dict = {}
    acc_dict = {}
    model.train()
    for epoch in range(epoch):
        perm = torch.randperm(len(X))
        train_loss = 0.0
        for i in range(0, len(X), batchsize):
            x = X[perm[i : i + batchsize]]
            y = Y[perm[i : i + batchsize]]
            output = model(x).squeeze()
            weight = model.dense_layer_1.weight.squeeze()
            loss = torch.mean(torch.clamp(1 - y * output, min=0))
            loss += step * (weight.t() @ weight) / 2.0
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += float(loss)
        loss_dict[epoch] = train_loss / len(X)
        acc = accuracy(X, Y, model)
        acc_dict[epoch] = acc

        if epoch%10 == 0:
            print(f"Epoch: {epoch+1}\tLoss: {train_loss / len(X)}\tAccuracy: {acc}")
            
            
    model.eval()
    return model, loss_dict, acc_dict

In [ ]:
torch.manual_seed(42)
model = SVM(40)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
epoch = 100
batchsize = 64
step = 0.01
model, loss_dict, acc_dict = train(x_train, y_train, optimizer, epoch, batchsize, step, model)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5)) 
x_coord, y_coord = zip(*(loss_dict.items()))
ax1.plot(x_coord, y_coord)
x_coord, y_coord = zip(*(acc_dict.items()))
ax2.plot(x_coord, y_coord)

In [ ]:
torch.manual_seed(42)
model = SVM(40)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
epoch = 100
batchsize = 64
step = 0.01
model, loss_dict, acc_dict = train(x_train, y_train, optimizer, epoch, batchsize, step, model)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5)) 
x_coord, y_coord = zip(*(loss_dict.items()))
ax1.plot(x_coord, y_coord)
x_coord, y_coord = zip(*(acc_dict.items()))
ax2.plot(x_coord, y_coord)

In [124]:
from pathlib import Path
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)
MODEL_NAME = "svm.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

In [125]:
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(), f=MODEL_SAVE_PATH) 

Saving model to: models\svm.pth


In [126]:
model = SVM(40)
model.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

<All keys matched successfully>

In [9]:
class EncryptedSVM:
    
    def __init__(self, torch_svm):
        self.weight = torch_svm.dense_layer_1.weight.data.tolist()[0]
        self.bias = torch_svm.dense_layer_1.bias.data.tolist()
        
    def forward(self, enc_x):
        enc_out = enc_x.dot(self.weight) + self.bias
        return enc_out
    
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)
    
    def encrypt(self, context):
        self.weight = ts.ckks_vector(context, self.weight)
        self.bias = ts.ckks_vector(context, self.bias)
        
    def decrypt(self, context):
        self.weight = self.weight.decrypt()
        self.bias = self.bias.decrypt()

In [13]:
encsvm = EncryptedSVM(model)

In [14]:
poly_mod_degree = 8192
coeff_mod_bit_sizes = [60, 40, 60]
ctx_eval = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
ctx_eval.global_scale = 2 ** 40
ctx_eval.generate_galois_keys()

In [37]:
# Function to calculate accuracy, recall, precision and f1 - score
def metrics_fn(y_true, y_pred, others=False):
    y_true = (y_true + 1) / 2
    y_pred = (y_pred + 1) / 2
            
    accuracy = torchmetrics.Accuracy(task="binary", num_classes=2)
    acc = accuracy(y_pred, y_true)
    
    if others==True:
        rec = torchmetrics.Recall(task="binary", num_classes=2)
        prec = torchmetrics.Precision(task="binary", num_classes=2)
        recall = rec(y_pred, y_true)
        precision = prec(y_pred, y_true)
        f1_score = 2*((precision*recall)/(precision+recall))
        del accuracy, rec, prec
        return acc, recall, precision, f1_score
    
    return acc

In [38]:
def encrypted_evaluation(model, enc_x_test, y_test):
    t_start = time.time()
    output_list = []
    correct = 0
    for enc_x, y in zip(enc_x_test, y_test):
        enc_out = model(enc_x)
        out = enc_out.decrypt()
        out = torch.tensor(out)
        out = torch.sign(out).float()
        output_list.append(out)
    t_end = time.time()
    print(f"Evaluated test set of {len(enc_x_test)} entries in {int(t_end - t_start)} seconds")
    accuracy = metrics_fn(torch.stack(output_list), y_test)
#     print(f"Score: {correct}/{len(enc_x_test)}")
    return accuracy, output_list

In [118]:
out = model(x_test_20).detach().squeeze().unsqueeze(dim=1)
out = torch.sign(out).float()
acc, rec, prec, f1 = metrics_fn(out, y_test_20, True)
print(f"Accuracy: {acc}\nRecall: {rec}\nPrecision: {prec}\nF1 - Score: {f1}")

Accuracy: 0.7743354439735413
Recall: 0.9646798968315125
Precision: 0.5695917010307312
F1 - Score: 0.7162664532661438


In [121]:
y_pred_final = []
parts = 20
one_part = y_test_20.shape[0] // parts
for i in range(1, parts+1):
    if i==1:
        j = one_part
        temp_x_test = x_test_20[:j,]
        temp_y_test = y_test_20[:j,]
    elif i==parts:
        j = one_part * (i-1)
        temp_x_test = x_test_20[j:,]
        temp_y_test = y_test_20[j:,]
    else:
        j = one_part * i
        if i == 1:
            k = one_part
        else:
            k = one_part * (i-1)
        temp_x_test = x_test_20[k:j,]
        temp_y_test = y_test_20[k:j,]
    
    t_start = time.time()
    enc_x_test = []
    print()
    print(f"Starting Batch {i} Encryption")
    for x in tqdm(range(len(temp_x_test))):
        enc_x_test.append(ts.ckks_vector(ctx_eval, temp_x_test[x].squeeze().tolist()))
    t_end = time.time()
    print(f"Encryption of the test set took {int(t_end - t_start)} seconds")
    encrypted_accuracy, output_list = encrypted_evaluation(eelr, enc_x_test, temp_y_test)
    print(f"Batch {i} Encrypted Accuracy: {encrypted_accuracy}")
    y_pred = torch.cat(output_list)
    y_pred = y_pred.numpy()
    y_pred_final.extend(y_pred)


Starting Batch 1 Encryption


100%|███████████████████████████████████████████████████████████████████████████████| 851/851 [00:04<00:00, 206.12it/s]


Encryption of the test set took 4 seconds
Evaluated test set of 851 entries in 7 seconds
Batch 1 Encrypted Accuracy: 0.7779083251953125

Starting Batch 2 Encryption


100%|███████████████████████████████████████████████████████████████████████████████| 851/851 [00:03<00:00, 234.94it/s]


Encryption of the test set took 3 seconds
Evaluated test set of 851 entries in 7 seconds
Batch 2 Encrypted Accuracy: 0.7591069340705872

Starting Batch 3 Encryption


...

Encryption of the test set took 3 seconds
Evaluated test set of 851 entries in 7 seconds
Batch 18 Encrypted Accuracy: 0.7955346703529358

Starting Batch 19 Encryption


100%|███████████████████████████████████████████████████████████████████████████████| 851/851 [00:03<00:00, 239.44it/s]


Encryption of the test set took 3 seconds
Evaluated test set of 851 entries in 7 seconds
Batch 19 Encrypted Accuracy: 0.7497062087059021

Starting Batch 20 Encryption


100%|███████████████████████████████████████████████████████████████████████████████| 861/861 [00:03<00:00, 239.27it/s]


Encryption of the test set took 3 seconds
Evaluated test set of 861 entries in 8 seconds
Batch 20 Encrypted Accuracy: 0.7781649231910706


In [123]:
out = model(x_test_20).detach().squeeze().unsqueeze(dim=1)
out = torch.sign(out).float()
acc, rec, prec, f1 = metrics_fn((torch.tensor(y_pred_final)).unsqueeze(dim=1), y_test_20, True)
print(f"Accuracy: {acc}\nRecall: {rec}\nPrecision: {prec}\nF1 - Score: {f1}")

Accuracy: 0.7668820023536682
Recall: 0.9485703110694885
Precision: 0.5627945065498352
F1 - Score: 0.7064477205276489
